## Load from CVS
The code in this notebook copies the cvs files for beaked whales from S3 to HDFS and then loads the data into 
a spark DataFrame.


In [3]:
from pyspark import SparkContext
sc = SparkContext(master=master_url)

from pyspark.sql import Row, SQLContext,DataFrame
from pyspark.sql.types import *

sqlContext = SQLContext(sc)

%pylab inline

!pip install pandas
!pip install scipy

import pandas as pd
import datetime as dt

from scipy.io import loadmat,savemat,whosmat

from string import split
from collections import Counter
import re
import numpy as np
from numpy import shape

from glob import glob

### Format of cvs files
|field name     | Description               | Data Type
|---------------|---------------------------|--------------
|0: time        | time of click             | string in datetime format `%Y-%m-%d %H:%M:%S.%f`
|1: species		| Initial species classification	        | 'str'
|2: site		| name of site		        | 'str'
|3: rec_no		| recording number		    | 'str'
|4: bout_i		| bout number		        | numpy.int64
|5: peak2peak	| peak to peak magnitude    | 			numpy.float64
|6: MSN	        |	wave form |		 an array of length 202
|208: MSP		|	spectra |	 an array of length 101  
|309: TPWS1		| 1 if click appears in TPWS1	| 	bool
|310: MD1		|	--- " ---	in MD1|	bool
|311: FD1	    |	--- " ---	in FD1|	bool
|312: TPWS2		| 1 if click appears in TPWS2	| 	bool
|313: MD2		|	--- " ---	in MD2|	bool
|314: FD2	    |	--- " ---	in FD2|	bool
|315: TPWS3		| 1 if click appears in TPWS3	| 	bool
|316: MD3		|	--- " ---	in MD3|	bool
|317: FD3	    |	--- " ---	in FD3|	bool
total number of fields= 318


### Parsing code

In [ ]:
#%%writefile lib/parse.py
import numpy as np
from pyspark.sql import Row, SQLContext,DataFrame
from pyspark.sql.types import *
import datetime as dt

class row_parser:
    date_format='%Y-%m-%d %H:%M:%S.%f'
    Fields=[('time', 'datetime'),
        ('species', 'str'),
        ('site', 'str'),
        ('rec_no', 'str'),
        ('bout_i', 'int'),
        ('peak2peak', 'float'),
        ('MSN', 'array',202),
        ('MSP', 'array',101),
        ('TPWS1', 'bool'),
        ('MD1', 'bool'),
        ('FD1', 'bool'),
        ('TPWS2', 'bool'),
        ('MD2', 'bool'),
        ('FD2', 'bool'),
        ('TPWS3', 'bool'),
        ('MD3', 'bool'),
        ('FD3', 'bool')]

    def __init__():
        #prepare date structure for parsing
        self.Parse_rules=[]
        index=0
        for field in Fields:
            _type=field[1]
            #print _type
            _len=1 # default length in terms of csv fields
            if _type =='array': 
                parser=parse_array
                _len=int(field[2])
            elif _type=='datetime': 
                parser=parse_date
            elif _type=='int': 
                parser=parse_int
            elif _type=='float': 
                parser=parse_float
            elif _type=='bool': 
                parser=parse_int
            elif _type=='str': 
                parser=parse_string
            else:
                print 'unrecognized type',_type
            rule={'name':field[0],
                  'start':index,
                  'end':index+_len,
                  'parser':parser}
            print field,rule
            self.Parse_rules.append(rule)
            index+=_len

        self.field_names=[a['name'] for a in self.Parse_rules]
        self.RowObject= Row(*self.field_names)


    def parse_date(s):
        #print 'date string="%s"'%s
        return dt.datetime.strptime(s,date_format)
    def parse_array(a):
        np_array=np.array([np.float64(x) for x in a])
        return packArray(np_array)
    def parse_int(s):
        return int(s)
    def parse_float(s):
        return float(s)
    def parse_string(s):
        return(s)

    def packArray(a):
        if type(a)!=np.ndarray:
            raise Exception("input to packArray should be numpy.ndarray. It is instead "+str(type(a)))
        return bytearray(a.tobytes())
    def unpackArray(x,data_type=np.int16):
        return np.frombuffer(x,dtype=data_type)


    def parse(row):
        items=row.split(',')
        D=[]
        for pr in self.Parse_rules:
            start=pr['start']
            end=pr['end']
            parser=pr['parser']
            if end-start==1:
                D.append(parser(items[start]))
            else:
                D.append(parser(items[start:end]))
        return self.RowObject(*D)



### Code for 

In [12]:
%cd /root/ipython/BeakedWhaleClassification/
%run /root/Credentials.ipynb

/root/ipython/BeakedWhaleClassification


In [13]:
s3helper.set_credential(AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY)

In [14]:
s3helper.open_bucket('while-classification')
s3helper.ls_s3()

[u'CVS',
 u'DT_Cuviers',
 u'DT_Gervais',
 u'GC_Cuviers',
 u'GC_Gervais',
 u'MC_Cuviers',
 u'MC_Gervais']

In [15]:
dirs=s3helper.ls_s3('CVS')
dirs[:10]

[u'CVS/DT.01.Cuviers',
 u'CVS/DT.01.Gervais',
 u'CVS/DT.02.Cuviers',
 u'CVS/DT.02.Gervais',
 u'CVS/DT.03.Cuviers',
 u'CVS/DT.03.Gervais',
 u'CVS/DT.04.Cuviers',
 u'CVS/DT.04.Gervais',
 u'CVS/DT.05.Cuviers',
 u'CVS/DT.05.Gervais']

In [16]:
from time import time

### Copy from S3 to HDFS

In [17]:
t1=time()
s3helper.s3_to_hdfs('CVS', 'CVS')
time()-t1

487.370493888855

### Read data into dataframe

In [18]:
from pyspark.sql import DataFrame
CVS_Data=sc.textFile("/CVS/")
RDD=CVS_Data.map(parse)

df=sqlContext.createDataFrame(RDD)

t0=time()
print df.cache().count()
print time()-t0

t0=time()
print df.count()
time()-t0

In [20]:
sc.stop()

6353182


0.8903360366821289